## Bellow the supported sigma categories for conversion to PQ

          - process_creation
          - file_event
          - file_change
          - file_rename
          - file_delete
          - image_load
          - pipe_creation
          - registry_add
          - registry_delete
          - registry_event
          - registry_set
          - dns_query 
          - dns
          - network_connection
          - firewall

### This is just a wrapper of the SentinelOne Backend project for Sigma, to include more categories the pipeline and backend file should be modified, fields should be manually map from the existent sigma fields to PQ fields.

In [2]:
import os
import yaml
from sigma.collection import SigmaCollection 
from sigma.configuration import SigmaConfiguration 
from sigma.backends.sentinelone_pq import SentinelOnePQBackend
from sigma.rule import SigmaRule

In [4]:
SUPPORTED_CATEGORIES = [ 'process_creation', 'file_event', 'file_change', 'file_rename', 'file_delete', 'image_load', 'pipe_creation', 'registry_add', 'registry_delete', 'registry_event', 'registry_set', 'dns_query', 'dns', 'network_connection', 'firewall' ]

In [12]:

def validate_yml_file(file_path):
    """
    Validate if the provided file path points to a .yml file.
    Parameters:
    file_path (str): The path of the file to validate.
    Returns:
    bool: True if the file is a .yml file, False otherwise.
    """
    try:
            if not os.path.isfile(file_path):
                print(f"File does not exist: {file_path}")
                return False
            if file_path.lower().endswith('.yml'):
                print(f"File is a valid .yml file: {file_path} \n")
                return True
            else:
                print(f"File is not a .yml file: {file_path} \n")
                return False
                
    except Exception as e:
        print(f"Error reading file: {e}")


def validatge_dir(file_path):
    try:
        files = os.listdir(file_path)
        yml_files = [file for file in files if file.endswith('.yml')]
        for file in yml_files:
                     print("[+] yml file found: "+file+ " Will try to covert to PQ \n ")
                     sigma = file_path+file
                     count_sigma_rules(sigma)
        return yml_files
    except Exception as e:
        print(f"An error ocurred: {e}")
        return[]

### parse sigma rules from the file
def count_sigma_rules(file_path):
    """
    Count the number of Sigma detection rules in the .yml file.
    Parameters:
    file_path (str): The path of the .yml file.
    Returns:
    int: The number of Sigma detection rules.
    """
    try:
        with open(file_path, 'r') as f:
            config = SigmaConfiguration()  # Default configuration
            rules = SigmaCollection.from_yaml(f, config)
        for rule in rules.rules: 
            category = rule.logsource.category
            title = rule.title
            if category not in SUPPORTED_CATEGORIES: 
                print("[-] Rule: "+title+" and Category: "+category+" is Not supported for conversion -_- \n")
            else:
                backend = SentinelOnePQBackend()
                print("[>] Rule: "+title+" to Powerquery: " )
                rulex = backend.convert_rule(rule)
                print(str(rulex).replace("[", "").replace("]", ""))
                print("\n") 
    except Exception as e:
        print(f"Error reading file: {e}")

if __name__ == "__main__":
    # Example file path - replace this with your actual file path in which sigma rules are located
    file_path = 'C:/Users/fabian.navarro/Documents/SigmaRules/To Covert/'
    # Validate the file
    if validate_yml_file(file_path):
        count_sigma_rules(file_path)
    else:
        validatge_dir(file_path)
        

File does not exist: C:/Users/fabian.navarro/Documents/SigmaRules/To Covert/
[+] yml file found: proc_creation_win_certutil_download_file_sharing_domains.yml Will try to covert to PQ 
 
[>] Rule: Suspicious File Downloaded From File-Sharing Website Via Certutil.EXE to Powerquery: 
'event.type="Process Creation" and (endpoint.os="windows" and ((tgt.process.image.path contains "\\certutil.exe" or OriginalFileName="CertUtil.exe") and (tgt.process.cmdline contains "urlcache " or tgt.process.cmdline contains "verifyctl ") and (tgt.process.cmdline contains ".githubusercontent.com" or tgt.process.cmdline contains "anonfiles.com" or tgt.process.cmdline contains "cdn.discordapp.com" or tgt.process.cmdline contains "cdn.discordapp.com/attachments/" or tgt.process.cmdline contains "ddns.net" or tgt.process.cmdline contains "dl.dropboxusercontent.com" or tgt.process.cmdline contains "ghostbin.co" or tgt.process.cmdline contains "glitch.me" or tgt.process.cmdline contains "gofile.io" or tgt.process